## Kravspecifikation för del 2

1. ✅ Börja använda basala **MLOps best-practices:**
    1. 📊 Ordna så att du har ett bra sätt att övervaka träningen, samt att spara (och hålla ordning på!) versioner av modellerna du tränar upp. (Tips: Foldrar, smart valda filnamn, etc).
    2. 💾 Kom ihåg att det troligen inte är den sista epokens checkpoint (sparad version) som är den bästa, p.g.a. overfitting om du kör många epochs. Du behöver själv se till att spara checkpoints lite då och då under träningen.
    3. 🗄️ Versionshantering av parametrar, kod, och checkpoints (modellens parametrar) blir snabbt väldigt viktigt när man börjar jobba seriöst med machine learning. Om man inte håller reda på vilken körning som gjorts med vilka parametrar, så blir det svårt att jämföra prestanda, vidareutveckla modellen senare, etc. Ingen vill ha en modell levererad där programmeraren inte minns hur den gjordes.
    4. 🚀 **Fördjupning/överkurs:** Vilka verktyg finns som kan hjälpa till att hålla ordning på versioner, körningar, resultat, hyper parameters, etc?
2. 🚀 **Fördjupning/överkurs:** Addera **performance metrics**, så att du kan se hur lång tid respektive tränings-körning tar. (En körning = Alla epochs av träning för en given modell med specificerad uppsättning hyper parameters).
3. ✅ Applicera lämpliga **data augmentation** methods för att artificiellt variera, och till och med “förstora” datamängden lite. Använd exempelvis skalning, rotation, färgvariation, brus, (spegelvändning?), etc. Här är lite repetition om data augmentation:
    1. Se avsnittet “[Uppgift 1 Perceptron för OCR](https://www.notion.so/Uppgift-1-Perceptron-fo-r-OCR-d9e945f1551341e88665775acacc0bb6?pvs=21)” längre ner i denna uppgift (del 3).
    2. Bra 10-minuters video om ämnet: [Pytorch-Data-Augmentation-using-Torchvision](https://youtu.be/Zvd276j9sZ8?si=9fGaNnoRzsHrvxB6)
    3. Observera att det inte är säkert att data augmentation leder till bättre resultat i alla lägen, eftersom det beror på detaljerna i vad man bygger, och vilken data man har.
4. ✅ Det är nu dags att testa CNN istället för FFN. Byt ut de första lagren i modellen till **convolutional layers** för att ta upp translationsinvarians i bilden. Sök själv upp vad man brukar ha ***direkt efter varje convolutional layer*** för att hålla ordning på antalet dimensioner till nästa lager .
    
    > (Svårt ord “translations-invarians”: translation=förflyttning, invarians=”samma oavsett” ⇒ translationsinvarians = ”spelar ingen roll var i bilden”)
    > 
5. ✅ Experimentera därefter med att addera ett till (eller flera) convolutional layer(s). Tanken är att testa **olika nät-arkitekturer**. Använd Google/Chat för initial gissning för modell-arkitektur, men sedan är det trial-and-error som gäller. Att fundera på:
    1. ⚖️ Hur påverkas resultatet av olika modell-arkitektur? Var noga med att inte jämföra modeller med stor skillnad i antal parameters, om aspekter som tex lagertyp ska utvärderas.
    2. 🤔 Vilken typ av features detekteras typiskt av de senare convolutional-lagren jämfört med det första convolutional-lagret?
        1. Hint: Videon från lektion om convolutional layers: [Convolutional-Neural-Networks-Explained](https://youtu.be/pj9-rr1wDhM?si=cjWmR5ets048WjfZ). (Självfallet kan du också plotta weight-matrices från din egen modell också om du vill!)
6. ✅ Applicera lämpliga **regularization methods** för att minimera risken för overfitting och göra cost-function-landskapet fördelaktigt för back-prop.
    1. Exempel på sådana metoder är: drop-out, weight-decay, noise injection, batch normalization, etc.
    2. Mer info om regularization: [understanding-regularization-with-pytorch](https://medium.com/analytics-vidhya/understanding-regularization-with-pytorch-26a838d94058)
7. 🚀 **Fördjupning:** Skapa en lista av kombinationer av hyper parameters, och låt datorn träna din modell på nytt för samtliga hyperparameter settings i din lista. Detta kallas för **hyper-parameter tuning**, d.v.s. att via trial-and-error hitta inställningar som funkar.
    - Enklast möjliga hyper-param-tuning: Gör en lista med t.ex. 10 rader (en per körning), där varje rad specificerar alla hyper-parameters som kan vara svårt att gissa optimalt värde på:
        - Antal neuroner för respektive lager
        - Typ av activation function
        - Vilka lager som ska vara convolutional, storlek på convolutional kernel, antal convolutional kernels per lager, etc.
        - Learning rate / hyper-params till optimizer ADAM
        - Drop-out level
        - Settings för olika typer av data augmentation (rotation, skalning, noise, etc)
    - Det finns även en mängd automatiserade typer av hyper-param-tuning. Se exempelvis: https://en.wikipedia.org/wiki/Hyperparameter_optimization

In [3]:
import os
import time
import logging
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Define the perceptron neural-network model
class Perceptron(nn.Module):
    # Define the constructor
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), # 32x32 with 1 padding for 28x28 input dimensions
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2) # 14x14 output dimensions
        )
        
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 14 * 14, 128), # 128 neurons in the hidden layer
            nn.ReLU(),
            nn.Linear(128, 10) # 10 neurons in the output layer
        )

    # Define the forward pass
    def forward(self, x):
        # Pass through the convolutional layers
        logits = self.conv_layers(x)

        # Flatten the output
        logits = logits.view(logits.size(0), -1)

        # Pass through the fully connected layers
        logits = self.fc_layers(logits)

        return logits


# Select device to run on
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

# Initialize the model
model = Perceptron().to(device)

# Set hyperparameters
learning_rate = 0.0001
num_epochs = 10
batch_size = 64
l1_lambda = 0.0001
l2_lambda = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

# Transformations for training data with data augmentation
training_transform = transforms.Compose([
    transforms.RandomRotation(10),  # Rotate by up to 10 degrees
    transforms.RandomAffine(0, scale=(0.8, 1.2)),  # Random scaling
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.1307,), (0.3081,)),  # Normalize with MNIST mean/std (pre-computed)
])

# Load the MNIST dataset
training_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=training_transform) # Apply transformations to the training data
testing_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Split training data into train and validation subsets
training_subset_size = int(0.8 * len(training_dataset))
validation_subset_size = len(training_dataset) - training_subset_size
training_subset, validation_subset = random_split(training_dataset, [training_subset_size, validation_subset_size])

# Create DataLoaders
train_loader = DataLoader(training_subset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_subset, batch_size=batch_size, shuffle=False)
testing_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False)

# Create a unique id and directory for the run
checkpoint_filename_prefix = 'checkpoint'
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join('models', f'run_{run_id}')
checkpoints_dir = os.path.join(run_dir, 'checkpoints')
os.makedirs('models', exist_ok=True)
os.makedirs(run_dir, exist_ok=True)
os.makedirs(checkpoints_dir, exist_ok=True)

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
log_file = os.path.join(run_dir, f'run_{run_id}_training.log')
fhandler = logging.FileHandler(filename=log_file, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

# Log hyperparameters
logger.info("=" * 100)
logger.info(f"Run ID: {run_id}")
logger.info(f"Training configuration:")
logger.info(f"Learning rate: {learning_rate}")
logger.info(f"Batch size: {batch_size}")
logger.info(f"L1 regularization: {l1_lambda}")
logger.info(f"L2 regularization: {l2_lambda}")
logger.info(f"Epochs: {num_epochs}")
logger.info(f"Optimizer: Adam")
logger.info(f"Loss function: CrossEntropyLoss")

# Training and validation loop
training_start_time = time.time()
best_val_loss = float('inf')
best_model_path = None
for epoch in range(num_epochs):
    # Cache the start time of the epoch
    epoch_start_time = time.time()

    # Training phase
    model.train()
    running_train_loss = 0.0
    for x, y in train_loader:
        # Move data to device
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        outputs = model(x)

        # Calculate base loss
        base_loss = criterion(outputs, y)
        
        # L1 regularization (L2 is handled by weight_decay)
        l1_reg = torch.tensor(0., device=device)
        for param in model.parameters():
            l1_reg += torch.norm(param, 1)
            
        # Total loss with L1 regularization
        loss = base_loss + l1_lambda * l1_reg

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_train_loss += loss.item()
    
    # Calculate average loss
    avg_train_loss = running_train_loss / len(train_loader)

    # Print training loss
    logger.info("="*100)
    logger.info(f"Epoch [{epoch+1}/{num_epochs}]")
    logger.info(f"Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in validation_loader:
            # Move data to device
            x, y = x.to(device), y.to(device)
            
            # Forward pass
            outputs = model(x)
            loss = criterion(outputs, y)
            running_val_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    
    # Calculate average loss and accuracy
    avg_val_loss = running_val_loss / len(validation_loader)
    val_accuracy = 100 * correct / total

    # Print validation loss and accuracy
    logger.info(f"Validation Loss: {avg_val_loss:.4f}")
    logger.info(f"Validation Accuracy: {val_accuracy:.2f}%")

    # Save the checkpoint
    checkpoint_filename = f'{checkpoint_filename_prefix}_epoch_{epoch+1}.pth'
    checkpoint_path = os.path.join(checkpoints_dir, checkpoint_filename)
    torch.save(model.state_dict(), checkpoint_path)

    # Update the best model if the current model has a lower validation loss
    if avg_val_loss < best_val_loss:
        best_model_path = checkpoint_path # Cache the path to the best model for later testing
        best_val_loss = avg_val_loss

    # Log the duration of the epoch
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    logger.info(f"Epoch {epoch+1} duration: {epoch_duration:.2f} seconds")

training_end_time = time.time()
training_duration = training_end_time - training_start_time
logger.info(f"Training duration: {training_duration:.2f} seconds")

# Get the best model for testing
model.load_state_dict(torch.load(best_model_path))

# Testing loop
testing_start_time = time.time()
model.eval()
running_test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for x, y in testing_loader:
        # Move data to device
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        running_test_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

# Calculate average loss and accuracy
avg_test_loss = running_test_loss / len(testing_loader)
test_accuracy = 100 * correct / total

# Log the results
testing_end_time = time.time()
testing_duration = testing_end_time - testing_start_time
total_run_time = testing_end_time - training_start_time
logger.info("="*100)
logger.info(f"Testing duration: {testing_duration:.2f} seconds")
logger.info(f"Total duration: {total_run_time:.2f} seconds")
logger.info("="*100)
logger.info(f"Best Model: {best_model_path}")
logger.info(f"Test Loss: {avg_test_loss:.4f}")
logger.info(f"Test Accuracy: {test_accuracy:.2f}%")
logger.info("="*100)

2025-04-23 18:58:19,597 - INFO - ====================================================================================================
2025-04-23 18:58:19,598 - INFO - Run ID: 20250423_185819
2025-04-23 18:58:19,598 - INFO - Training configuration:
2025-04-23 18:58:19,599 - INFO - Learning rate: 0.0001
2025-04-23 18:58:19,599 - INFO - Batch size: 64
2025-04-23 18:58:19,599 - INFO - L1 regularization: 0.0001
2025-04-23 18:58:19,600 - INFO - L2 regularization: 0.001
2025-04-23 18:58:19,600 - INFO - Epochs: 10
2025-04-23 18:58:19,601 - INFO - Optimizer: Adam
2025-04-23 18:58:19,601 - INFO - Loss function: CrossEntropyLoss
2025-04-23 18:58:30,657 - INFO - ====================================================================================================
2025-04-23 18:58:30,658 - INFO - Epoch [1/10]
2025-04-23 18:58:30,659 - INFO - Training Loss: 0.8305
2025-04-23 18:58:33,147 - INFO - Validation Loss: 0.3006
2025-04-23 18:58:33,148 - INFO - Validation Accuracy: 91.67%
2025-04-23 18:58:33,1